In [10]:
# !pip install huggingface_hub

In [15]:
# !pip install datasets

In [3]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
import chromadb

In [12]:
# Checking MPS availability for Apple Silicon GPU acceleration
mps_available = torch.backends.mps.is_built() and torch.backends.mps.is_available()
device = "mps" if mps_available else "cpu"
print("Using device:", device)

Using device: mps


In [11]:
from huggingface_hub import HfApi

dataset_url = "SathvikVeerapaneni7/CineAI_Dataset"
api = HfApi()

# List files in the dataset repository
files = api.list_repo_files(repo_id=dataset_url, repo_type="dataset")
print(f"Files in the Hugging Face dataset '{dataset_url}':")
for f in files:
    print(f)

Files in the Hugging Face dataset 'SathvikVeerapaneni7/CineAI_Dataset':
.gitattributes
README.md
parquet_files/cast_df_clean.parquet
parquet_files/crew_df_clean.parquet
parquet_files/details_df_clean.parquet
parquet_files/recommendations_df_clean.parquet
parquet_files/watch_providers_df_clean.parquet


In [ ]:
from datasets import load_dataset

# sample Loading a single Parquet file
dataset = load_dataset(
    "parquet",
    data_files={"train": "https://huggingface.co/datasets/SathvikVeerapaneni7/CineAI_Dataset/resolve/main/parquet_files/details_df_clean.parquet"},
    split="train"
)

# Convert to DataFrame for processing if needed
import pandas as pd
details_df = pd.DataFrame(dataset)
print("Loaded details_df from HF:", details_df.shape)
details_df.head()

details_df_clean.parquet:   0%|          | 0.00/51.1M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loaded details_df from HF: (230586, 8)


,movie_id,title,overview,release_date,runtime,original_language,popularity,genres_str
0,98.0,Gladiator,"In the year 180, the death of Emperor Marcus A...",2000-05-04,155.0,en,497.649,['Action' 'Drama' 'Adventure']
1,8871.0,How the Grinch Stole Christmas,The Grinch decides to rob Whoville of Christma...,2000-11-17,104.0,en,177.539,['Family' 'Comedy' 'Fantasy']
2,7443.0,Chicken Run,The creators of Wallace & Gromit bring you an ...,2000-06-23,84.0,en,62.638,['Animation' 'Comedy' 'Family']
3,9532.0,Final Destination,After a teenager has a terrifying vision of hi...,2000-03-17,98.0,en,54.097,['Horror']
4,77.0,Memento,Leonard Shelby is tracking down the man who ra...,2000-10-11,113.0,en,95.374,['Mystery' 'Thriller']
